In [ ]:
%pylab inline

# JSAnimation import available at https://github.com/jakevdp/JSAnimation
from JSAnimation.JSAnimation import IPython_display
from matplotlib import animation

import numpy as np

from intern.remote.boss import BossRemote
from intern.resource.boss.resource import *

## Setup `intern` Remote Instance

Create an instance of the Boss Python SDK intern.  You need to provide configuration details to the library.

- Get your API Token: https://api.theboss.io/token
- Update `./demo.cfg` to contain your token

In [ ]:
rmt = BossRemote('./demo.cfg')

## Create a Channel Resource

We interact with the Boss via "resources" which represent how data is organized in the Boss.  

`Channels` store image data.  Let's get an existing channel that has been loaded with a small 4096x4096x48 chunk of data from the Princeton Waypoint

In [ ]:
ch = ChannelResource("anno", "demo8", "example")
ch = rmt.get_project(ch)

## Reserve ID Service

The Boss provides an atomic ID Reservation service, in case you want to upload results through the cutout service.

Simply provide the number of IDs to reservce, and intern returns the starting ID of the block you requested. If you do not upload any data using the IDs, they will go unused.

In [ ]:
first_id = rmt.reserve_ids(ch, 20)
print("Reserved ID block: [{}, {}]".format(first_id, first_id + 20))

first_id = rmt.reserve_ids(ch, 100)
print("Reserved ID block: [{}, {}]".format(first_id, first_id + 100))

## IDs in a region

You can use `intern` to query the Boss for annotation IDs inside a bounding box

In [ ]:
# ids in region aligned to underlying cuboids
x_rng = [512, 1024]
y_rng = [512, 1024]
z_rng = [16, 32]
id_list = rmt.get_ids_in_region(ch, 0, x_rng, y_rng, z_rng)
print("IDs in Region: {}".format(id_list))

In [ ]:
# ids in region unaligned from underlying cuboids
x_rng = [512, 800]
y_rng = [512, 700]
z_rng = [16, 18]
id_list = rmt.get_ids_in_region(ch, 0, x_rng, y_rng, z_rng)
print("IDs in Small Unaligned Region: {}".format(id_list))

## Bounding Box

You can use intern to retrieve the bounding box around an annotation ID. Currently only "loose" bounding boxes are supported, which are aligned to cuboid boundaries (512x512x16). In the near future "tight" bounding boxes will be available that minimally enclose and ID

In [ ]:
print("Get bounding box for ID: {}".format(id_list[0]))
bb_loose = rmt.get_bounding_box(ch, 0, id_list[0], 'loose')
print(bb_loose)

## Filter Cutout by ID

You can now provide an optional list of IDs to the `get_cutout` method, and the returned cutout will be filtered to only include those IDs on the server.

In [ ]:
# normal cutout
cutout = rmt.get_cutout(ch, 0,
                        [512, 1024],
                        [512, 1024],
                        [0, 32])

In [ ]:
# Plot
fig = plt.figure()
im = plt.imshow(cutout[0,:,:], cmap=plt.cm.jet, interpolation='bicubic') 

def init():
    im.set_data(cutout[0,:,:])

def animate(i): 
    im.set_data(cutout[i,:,:])
    return im

animation.FuncAnimation(fig, animate, init_func=init, frames=cutout.shape[0],
                               interval=75)

In [ ]:
# filtered cutout
filtered_cutout = rmt.get_cutout(ch, 0,
                        [512, 1024],
                        [512, 1024],
                        [0, 32],
                        id_list=[int(id_list[0])])

In [ ]:
# Plot
fig = plt.figure()
im = plt.imshow(filtered_cutout[0,:,:], cmap=plt.cm.jet, interpolation='bicubic') 

def init():
    im.set_data(filtered_cutout[0,:,:])

def animate(i): 
    im.set_data(filtered_cutout[i,:,:])
    return im

animation.FuncAnimation(fig, animate, init_func=init, frames=filtered_cutout.shape[0],
                               interval=75)

In [ ]:
# filtered cutout on an ID that doesn't exist should return an empty matrix
filtered_cutout = rmt.get_cutout(ch, 0,
                        [512, 1024],
                        [512, 1024],
                        [0, 32],
                        id_list=[156146464])
print(sum(filtered_cutout))